## CMSC461-01: Database Management Systems, Fall 2018
### Project 1

In [1]:
# import some packages and define two helper functions python functions to 
# connect to DB, and execute a SQL statement and show its results

import mysql.connector
import pandas as pd
import getpass
import datetime

def getConnectionToDB(db_user, db_password, db_name='cuber'):
    config = { 'user': db_user, 'password': db_password, 
              'host': 'localhost', 'port': 3306, 'database': db_name, 
              'raise_on_warnings': True                    
    }
    db_connection = mysql.connector.connect(**config)
    return db_connection

def executeQueryAndShowResults(query, notuples=0):
    # execute a SQL query and print the first notuples rows in the result
    # notuples is 5 by default; use a non-positive integer for all tuples
    # in the result
    df = pd.read_sql(query, con=db_connection)
    if notuples <= 0:
        notuples = df.shape[0]
    print(df.head(notuples))

In [2]:
db_user = getpass.getpass('Enter username')
db_password = getpass.getpass('Enter password')
db_connection = getConnectionToDB(db_user, db_password)

Enter username········
Enter password········


#### 1. List the names of all the drivers.

In [3]:
query = '''
SELECT first_name, last_name 
FROM driver
'''
executeQueryAndShowResults(query)

   first_name last_name
0       Sandy     Smith
1       Clare      Boyd
2        Emma       Day
3      Serina      Horn
4      Daniel    Holmes
5         Lee    Melend
6        Macy    Forbes
7        Bert    Simson
8        Eric     Smith
9        Dana       Tay
10       Carl     Sweet
11     Justin   Garrett


#### 2. Find the total customer-times-miles during a given date.

In [4]:
# change YYYY-MM-DD to test a different date
query = '''
SELECT SUM(DISTINCT T.total_dist) as date_miles_total
FROM trip as T, trip_traj as S
WHERE (S.timestamp BETWEEN '2018-12-08' AND '2018-12-09') AND (T.trip_id = S.trip_id)
'''
executeQueryAndShowResults(query)

   date_miles_total
0              14.0


#### 3. Find the most “active” (in terms of number of trips) driver during a given month. 

In [5]:
# change YYYY-MM-DD to test a different month
query = '''
SELECT T.driver_id, COUNT(DISTINCT T.trip_id) as december_trip_count
FROM trip as T, trip_traj as S
WHERE (S.timestamp >= '2018-12-00') AND (T.trip_id = S.trip_id) 
GROUP BY driver_id
ORDER by december_trip_count DESC
LIMIT 1
'''
executeQueryAndShowResults(query)

  driver_id  december_trip_count
0       d10                    2


#### 4. Find the “most” valuable customer (in terms of dollars charged).

In [6]:
query = '''
SELECT T.cust_id, SUM(DISTINCT T.amount) as amount_charged
FROM payment as T
WHERE T.amount < 0
GROUP BY cust_id
ORDER by amount_charged ASC
LIMIT 1
'''
executeQueryAndShowResults(query)

  cust_id  amount_charged
0      c7          -130.0


#### 5. Find the balance owed by a given customer.

In [7]:
query = '''
SELECT T.cust_id, SUM(DISTINCT T.amount) as balance_owed
FROM payment as T
WHERE (T.amount < 0) AND cust_id = 'c3'
'''
executeQueryAndShowResults(query)

  cust_id  balance_owed
0      c3         -15.0


#### 6. Find the vehicles that traveled more miles in a given month than the previous month.

In [8]:
# change YYYY-MM-DD to test a different month

# DECEMBER:
query = '''
SELECT T.driver_id, SUM(DISTINCT T.total_dist) as december_miles
FROM trip as T, trip_traj as S
WHERE (S.timestamp >= '2018-12-00') AND (T.trip_id = S.trip_id) 
GROUP BY driver_id
ORDER by december_miles DESC
'''
executeQueryAndShowResults(query)

# NOVEMBER:
query = '''
SELECT T.driver_id, SUM(DISTINCT T.total_dist) as november_miles
FROM trip as T, trip_traj as S
WHERE (S.timestamp BETWEEN '2018-11-00' AND '2018-12-00') AND (T.trip_id = S.trip_id) 
GROUP BY driver_id
ORDER by november_miles DESC
'''
executeQueryAndShowResults(query)

  driver_id  december_miles
0       d10            65.0
1        d6            13.0
2        d3            10.0
3        d7             1.0
4        d1             1.0
  driver_id  november_miles
0        d6            55.0
1        d5            50.0
2        d2            20.0
3       d11            10.0
4        d1             2.0


#### 7. Find the most popular starting and ending stops in a given week.

In [9]:
# change YYYY-MM-DD to test a different week

# STARTING STOP:
query = '''
SELECT latitude, longitude, COUNT(latitude), COUNT(longitude)
FROM trip_traj 
WHERE (timestamp BETWEEN '2018-12-02' AND '2018-12-09')
GROUP BY latitude, longitude
ORDER by COUNT(latitude) DESC
LIMIT 1
'''
executeQueryAndShowResults(query)

# ENDING STOP:
query = '''
SELECT latitude, longitude, COUNT(latitude), COUNT(longitude)
FROM trip_traj 
WHERE (timestamp BETWEEN '2018-12-02' AND '2018-12-09') AND (dist_from_last = 0)
GROUP BY latitude, longitude
ORDER by COUNT(latitude) DESC
LIMIT 1
'''
executeQueryAndShowResults(query)

   latitude  longitude  COUNT(latitude)  COUNT(longitude)
0  39.40431  -76.59951                4                 4
   latitude  longitude  COUNT(latitude)  COUNT(longitude)
0  38.98776   -76.9139                1                 1


In [ ]:
# close the connection to the DB server
db_connection.close()
print(datetime.datetime.now())
!whoami
!hostname